In [1]:
import glob
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
from scipy import ndimage
import pickle
import nibabel as nib
import random
from scipy.ndimage.interpolation import zoom


from torchvision import transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import pysgmcmc

In [2]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def read_mutant_txt(path):
    name_list = []
    fo = open(path)
    for line in fo:
        striped_line = line.strip('\n')
        if striped_line != '':
            name_list.append(striped_line)
    return name_list


In [3]:
mutant_names = read_mutant_txt('mutant_imgs.txt')
data_base_path = '/scratch/zq415/grammar_cor/mutant_detect/data'
data_folder_list = ['20180419_newdata_nii_with_filtered', 'new_data_20180522_nii', 'organized_data_nii']

In [4]:
save_name = 'All_data_112_64_64.pickle'
with open(os.path.join(os.getcwd(),'data',save_name), "rb") as input_file:
    all_train_data = pickle.load(input_file)

In [5]:
mutant_group = [(9,10), (12,13,14), (16,17,18,19), (36,37), (38,39), (42,43), (44,45,46,47), (48,49,50,51), (52,53), (54,55),
(56,57,58), (59,60,61), (63,64), (66,67,68), (69,70,71), (73,74), (75,76), (77,78,79), (80,81,82,83,84,85,86,87),
(89,90), (91,92), (93,94), (95,96,97), (98,99), (100,101,102)]

group_list = []
for one_group in mutant_group:
    for ii in range(len(one_group)):
        group_list.append(one_group[ii])
single_mutant = [i for i in range(len(mutant_names)) if i not in group_list]

test_mut_names = []
for i in range(1,len(mutant_group),6):
    for ii in range(len(mutant_group[i])):
        test_mut_names.append(mutant_names[mutant_group[i][ii]])

for i in range(1,len(single_mutant),6):
    test_mut_names.append(mutant_names[single_mutant[i]])
    
print(len(test_mut_names),test_mut_names)


18 ['20170207_En1_E13_E3_Mut', '20170207_En1_E13_E3c_Mut', '20170207_En1_E13_E3d_Mut', '20161206_En1_E13_E3a', '20161206_En1_E13_E3b', '20161206_En1_E13_E3c', '20161206_En1_E13_E3d', '20180131_En1_E12_E5a_Mut', '20180131_En1_E12_E5b', '20180131_En1_E12_E5c_Mut', '20180202_En1_E14_E3a_Mut', '20180202_En1_E14_E3b_Mut', '20171211_En1_E10_E1a', '20170619_En1_E12_E3a', '20170706_En1_E13_E11a_Mut_reg', '20171009_En1_E12_E10a', '20171211_En1_E13_E5a_Mut_Ext', '20180201_En1_M2_E13_E4a']


In [6]:
train_data = []
test_data = []

for i in range(len(all_train_data)):
    if all_train_data[i][2] in mutant_names:
        if all_train_data[i][2] in test_mut_names:
            test_data.append((all_train_data[i][3]-0.5, 0 ))
        else:
            train_data.append((all_train_data[i][3]-0.5,0))
    else:
        random.seed(i*8)
        if random.uniform(0,1) < 0.16:
            test_data.append((all_train_data[i][3]-0.5,1))
        else:
            train_data.append((all_train_data[i][3]-0.5,1))

print(len(test_data))
print(len(train_data))


96
471


In [7]:
#img_nft = nib.Nifti1Image(np.squeeze(all_train_data[193]),np.eye(4))
#img_save_data_path = './img/mul_img.nii'
#nib.save(img_nft,img_save_data_path)

In [8]:
from torch.utils.data import Dataset, DataLoader

class Mouse_sub_volumes(Dataset):
    """Mouse sub-volumes BV dataset."""

    def __init__(self, all_data , transform=None):
        """
        Args:
            all_whole_volumes: Contain all the padded whole BV volumes as a dic
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.all_data = all_data
        self.transform = transform
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self, num):
        
        current_img, label = self.all_data[num]
        
        img = np.float32(current_img[np.newaxis,...])
        sample = {'image': img, 'label': label}
        return sample

In [9]:
class Flip(object):
    
    """
    Flip the image for data augmentation, but prefer original image.
    """
    
    def __init__(self,ori_probability=0.20):
        self.ori_probability = ori_probability

    def __call__(self, sample):
        if random.uniform(0,1) < self.ori_probability:
            return sample
        else:
            img, label = sample['image'], sample['label']
            random_choise1=random.choice([1,2,3,4,5,6,7,8])
            img[0,...] = {1: lambda x: x,
                          2: lambda x: x[::-1,:,:],
                          3: lambda x: x[:,::-1,:],
                          4: lambda x: x[:,:,::-1],
                          5: lambda x: x[::-1,::-1,:],
                          6: lambda x: x[::-1,:,::-1],
                          7: lambda x: x[:,::-1,::-1],
                          8: lambda x: x[::-1,::-1,::-1]
                          }[random_choise1](img[0,...])
        return {'image': img, 'label': label}

In [10]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class VGG_net(nn.Module):
    def __init__(self,conv_drop_rate=0.10,linear_drop_rate=0.2):
        super(VGG_net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=12, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv1_bn = nn.BatchNorm3d(12)
        self.conv2 = nn.Conv3d(in_channels=12, out_channels=12, kernel_size=3,stride=1,padding=2, dilation=2)
        self.conv2_bn = nn.BatchNorm3d(12)
        self.pool1 = nn.MaxPool3d(2, 2)
        self.dropout1 = nn.Dropout3d(conv_drop_rate)
        
        self.conv3 = nn.Conv3d(in_channels=12, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv3_bn = nn.BatchNorm3d(24)
        self.conv4 = nn.Conv3d(in_channels=24, out_channels=24, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv4_bn = nn.BatchNorm3d(24)
        self.pool2 = nn.MaxPool3d(2, 2)
        self.dropout2 = nn.Dropout3d(conv_drop_rate)
        
        self.conv5 = nn.Conv3d(in_channels=24, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv5_bn = nn.BatchNorm3d(48)
        self.conv6 = nn.Conv3d(in_channels=48, out_channels=48, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv6_bn = nn.BatchNorm3d(48)
        self.pool3 = nn.MaxPool3d(2, 2)
        self.dropout3 = nn.Dropout3d(conv_drop_rate)
        
        self.conv7 = nn.Conv3d(in_channels=48, out_channels=96, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv7_bn = nn.BatchNorm3d(96)
        self.conv8 = nn.Conv3d(in_channels=96, out_channels=96, kernel_size=3,stride=1, padding=2,dilation=2)
        self.conv8_bn = nn.BatchNorm3d(96)
        self.pool4 = nn.AdaptiveAvgPool3d((1,1,1))
        self.dropout4 = nn.Dropout3d(conv_drop_rate)
        
        
        self.fc1 = nn.Linear(96, 2)
        
    def forward(self, x):
        x = self.conv1_bn(F.relu(self.conv1(x)))
        x = self.dropout1(self.pool1(self.conv2_bn(F.relu(self.conv2(x)))))
        
        x = self.conv3_bn(F.relu(self.conv3(x)))
        x = self.dropout2(self.pool2(self.conv4_bn(F.relu(self.conv4(x)))))
        
        x = self.conv5_bn(F.relu(self.conv5(x)))
        x = self.dropout3(self.pool3(self.conv6_bn(F.relu(self.conv6(x)))))
        
        x = self.conv7_bn(F.relu(self.conv7(x)))
        x = self.dropout4(self.pool4(self.conv8_bn(F.relu(self.conv8(x)))))
        
        x = x.view(-1, 96)
        #x = self.dropout5(self.fc1_bn(F.relu(self.fc1(x))))
        x = self.fc1(x)
        return x


In [19]:
!pip install git+https://github.com/MFreidank/pysgmcmc@pytorch

  Cloning https://github.com/MFreidank/pysgmcmc (to revision pytorch) to /state/partition1/job-5988547/pip-req-build-_6j_xeqz
Branch pytorch set up to track remote branch pytorch from origin.
Switched to a new branch 'pytorch'
  Running setup.py bdist_wheel for pysgmcmc ... done
  Stored in directory: /state/partition1/job-5988547/pip-ephem-wheel-cache-g_b2tfz4/wheels/0d/08/b8/56459f78d37ad065ad2774ce2b8839bbda57e519e468fc3ed4
Successfully built pysgmcmc
You are using pip version 18.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [24]:
from pysgmcmc.optimizers.sgld import SGLD

In [25]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)

optimizer = SGLD(net.parameters())

In [11]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    running_loss = 0.0
    for i_batch, sample_batched in enumerate(train_loader):
        inputs, labels = sample_batched['image'], sample_batched['label']  
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        i_batch += 1
        if i_batch % 10 == 0:
            print("epoch {}, batch {}, current loss {}".format(epoch+1,i_batch,running_loss/10))
            running_loss = 0.0

def test(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy()))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

def get_confusion_matrix(true_predicted_labels):
    cross_table = np.zeros([2,2])
    mut_to_nor = []
    nor_to_mul = []
    test_dic = true_predicted_labels
    for i in range(len(test_dic)):
        if test_dic[i][0] ==0 and test_dic[i][1] ==0:
            cross_table[0,0] += 1
        elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
            cross_table[0,1] += 1
            mut_to_nor.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
            cross_table[1,0] += 1
            nor_to_mul.append(i)
        elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
            cross_table[1,1] += 1
    print(cross_table)
    print(mut_to_nor)
    print(nor_to_mul)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

num_epochs = 350
criterion = nn.CrossEntropyLoss(weight=torch.FloatTensor([3.5,1.0]).to(device))

optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9, weight_decay=0.00001)
print('choose SGD as optimizer')
#optimizer = optim.Adam(net.parameters(), lr=args.lr*10, weight_decay=0.00001)
#print('choose Adam as optimizer')

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)

for epoch in range(num_epochs):
    scheduler.step()
    
    Mouse_dataset = Mouse_sub_volumes(train_data, transform=transforms.Compose([Flip()]))
    dataloader = DataLoader(Mouse_dataset, batch_size=12, shuffle=True, num_workers=4, drop_last = True)
    train(net, device, dataloader, optimizer, criterion, epoch)
    if (epoch + 1) % 10 == 0:
        print('epoch {} train accuracy: '.format(epoch+1))
        train_Mouse_dataset = Mouse_sub_volumes(train_data)
        train_dataloader = DataLoader(train_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        train_dic = test(net, device, train_dataloader)
        get_confusion_matrix(train_dic)
        
        print("-------------------")
        print('epoch {} test accuracy: '.format(epoch+1))
        test_Mouse_dataset = Mouse_sub_volumes(test_data)
        test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)
        test_dic = test(net, device, test_dataloader)
        get_confusion_matrix(test_dic)
        
        torch.save(net.state_dict(), './model/mut_clas_2019_01_21_e{}_global2.pth'.format(epoch+1))

There are 495374 parameters in the model
choose SGD as optimizer
epoch 1, batch 10, current loss 0.6956422030925751
epoch 1, batch 20, current loss 0.6869348227977753
epoch 1, batch 30, current loss 0.6631369709968566
epoch 2, batch 10, current loss 0.6257618606090546
epoch 2, batch 20, current loss 0.6146553337574006
epoch 2, batch 30, current loss 0.633811867237091
epoch 3, batch 10, current loss 0.5761816740036011
epoch 3, batch 20, current loss 0.46665129661560056
epoch 3, batch 30, current loss 0.5657233476638794
epoch 4, batch 10, current loss 0.4539719194173813
epoch 4, batch 20, current loss 0.39927015602588656
epoch 4, batch 30, current loss 0.5526776939630509
epoch 5, batch 10, current loss 0.41829574182629586
epoch 5, batch 20, current loss 0.5022016152739525
epoch 5, batch 30, current loss 0.3793440401554108
epoch 6, batch 10, current loss 0.29261579662561416
epoch 6, batch 20, current loss 0.30306785106658934
epoch 6, batch 30, current loss 0.22168605253100396
epoch 7, bat

epoch 42, batch 30, current loss 0.006617568311048672
epoch 43, batch 10, current loss 0.005553472292376682
epoch 43, batch 20, current loss 0.006391102919587865
epoch 43, batch 30, current loss 0.010187715862412006
epoch 44, batch 10, current loss 0.01822757933405228
epoch 44, batch 20, current loss 0.01822001617401838
epoch 44, batch 30, current loss 0.028836075065191834
epoch 45, batch 10, current loss 0.00590320301707834
epoch 45, batch 20, current loss 0.013996994990156964
epoch 45, batch 30, current loss 0.020867846440523863
epoch 46, batch 10, current loss 0.005766959686297923
epoch 46, batch 20, current loss 0.03715946272131987
epoch 46, batch 30, current loss 0.013497328619996552
epoch 47, batch 10, current loss 0.01804182767518796
epoch 47, batch 20, current loss 0.01943927933316445
epoch 47, batch 30, current loss 0.0051861820160411295
epoch 48, batch 10, current loss 0.012055339472135529
epoch 48, batch 20, current loss 0.03538017862010747
epoch 48, batch 30, current loss 0

epoch 85, batch 20, current loss 0.002388770259858575
epoch 85, batch 30, current loss 0.008559757335751783
epoch 86, batch 10, current loss 0.009992879352648743
epoch 86, batch 20, current loss 0.009805998866795562
epoch 86, batch 30, current loss 0.0020526052270724905
epoch 87, batch 10, current loss 0.01956308233930031
epoch 87, batch 20, current loss 0.023049670239561237
epoch 87, batch 30, current loss 0.0030625476269051433
epoch 88, batch 10, current loss 0.007672017259028508
epoch 88, batch 20, current loss 0.006354524944617879
epoch 88, batch 30, current loss 0.0022551067297172265
epoch 89, batch 10, current loss 0.007615884681581519
epoch 89, batch 20, current loss 0.04131485071411589
epoch 89, batch 30, current loss 0.009122251775988844
epoch 90, batch 10, current loss 0.0031225039609125817
epoch 90, batch 20, current loss 0.0021590915326669346
epoch 90, batch 30, current loss 0.0007471209828509018
epoch 90 train accuracy: 
total_num:471, test accuracy:1.0, positive_acc:1.0, 

epoch 127, batch 10, current loss 0.003338714539495413
epoch 127, batch 20, current loss 0.0007178973013651557
epoch 127, batch 30, current loss 0.008238878671545536
epoch 128, batch 10, current loss 0.0010565154734649695
epoch 128, batch 20, current loss 0.0013237244813353755
epoch 128, batch 30, current loss 0.0005623671710054623
epoch 129, batch 10, current loss 0.0017398396150383633
epoch 129, batch 20, current loss 0.0005143883850905695
epoch 129, batch 30, current loss 0.0010936846818367485
epoch 130, batch 10, current loss 0.0016932637197896838
epoch 130, batch 20, current loss 0.001250858411731315
epoch 130, batch 30, current loss 0.0008873977392795495
epoch 130 train accuracy: 
total_num:471, test accuracy:1.0, positive_acc:1.0, negative_acc:1.0
[[  86.    0.]
 [   0.  385.]]
[]
[]
-------------------
epoch 130 test accuracy: 
total_num:96, test accuracy:0.9479166666666666, positive_acc:0.9743589743589743, negative_acc:0.8333333333333334
[[ 15.   3.]
 [  2.  76.]]
[0, 88, 90]


epoch 167, batch 30, current loss 0.0006327678349407507
epoch 168, batch 10, current loss 0.00013065386556263548
epoch 168, batch 20, current loss 0.0023369872567855056
epoch 168, batch 30, current loss 0.0004822729597435682
epoch 169, batch 10, current loss 0.0005331634672984364
epoch 169, batch 20, current loss 0.00027905124134122163
epoch 169, batch 30, current loss 9.895631737890653e-05
epoch 170, batch 10, current loss 0.0008335155676832074
epoch 170, batch 20, current loss 0.0020967313379514963
epoch 170, batch 30, current loss 0.00033171818613482175
epoch 170 train accuracy: 
total_num:471, test accuracy:1.0, positive_acc:1.0, negative_acc:1.0
[[  86.    0.]
 [   0.  385.]]
[]
[]
-------------------
epoch 170 test accuracy: 
total_num:96, test accuracy:0.9583333333333334, positive_acc:0.9743589743589743, negative_acc:0.8888888888888888
[[ 16.   2.]
 [  2.  76.]]
[0, 90]
[32, 38]
epoch 171, batch 10, current loss 0.00032361195735575166
epoch 171, batch 20, current loss 0.00164918

epoch 208, batch 20, current loss 0.0009519442610326223
epoch 208, batch 30, current loss 0.00036354849789859144
epoch 209, batch 10, current loss 0.00021243767187115737
epoch 209, batch 20, current loss 0.001425344687413599
epoch 209, batch 30, current loss 0.0004905563002466807
epoch 210, batch 10, current loss 0.0007825203029824479
epoch 210, batch 20, current loss 0.00021382989498306416
epoch 210, batch 30, current loss 0.001093894863879541
epoch 210 train accuracy: 
total_num:471, test accuracy:1.0, positive_acc:1.0, negative_acc:1.0
[[  86.    0.]
 [   0.  385.]]
[]
[]
-------------------
epoch 210 test accuracy: 
total_num:96, test accuracy:0.9479166666666666, positive_acc:0.9743589743589743, negative_acc:0.8333333333333334
[[ 15.   3.]
 [  2.  76.]]
[0, 88, 90]
[32, 38]
epoch 211, batch 10, current loss 0.002622373535996303
epoch 211, batch 20, current loss 0.0003258159807046468
epoch 211, batch 30, current loss 0.00045782879960825084
epoch 212, batch 10, current loss 0.0002870

epoch 249, batch 10, current loss 0.00024165689537767321
epoch 249, batch 20, current loss 0.06753499287942759
epoch 249, batch 30, current loss 0.0003938825509976596
epoch 250, batch 10, current loss 0.0020579210311552744
epoch 250, batch 20, current loss 0.0015053903483931209
epoch 250, batch 30, current loss 0.00044351248907332775
epoch 250 train accuracy: 
total_num:471, test accuracy:1.0, positive_acc:1.0, negative_acc:1.0
[[  86.    0.]
 [   0.  385.]]
[]
[]
-------------------
epoch 250 test accuracy: 
total_num:96, test accuracy:0.9479166666666666, positive_acc:0.9487179487179487, negative_acc:0.9444444444444444
[[ 17.   1.]
 [  4.  74.]]
[90]
[16, 32, 37, 38]
epoch 251, batch 10, current loss 0.00037665629461116625
epoch 251, batch 20, current loss 0.0014377789422724164
epoch 251, batch 30, current loss 4.813689292859635e-05
epoch 252, batch 10, current loss 0.0003581197485800658
epoch 252, batch 20, current loss 0.00038807898490631485
epoch 252, batch 30, current loss 0.00199

Process Process-1219:
Process Process-1218:
Process Process-1220:
Process Process-1217:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/share/apps/python3/3.6.3/intel/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not sel

KeyboardInterrupt: 

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = VGG_net()
#net.apply(weight_init)
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    net = nn.DataParallel(net)
net.to(device)
print("There are {} parameters in the model".format(count_parameters(net)))

There are 495374 parameters in the model


In [58]:
def test_with_probability(model, device, test_loader):
    model.eval()
    correct_num = 0
    total_num = 0
    positive_correct=0
    positive_num=0
    negative_correct=0
    negative_num=0
    
    true_predicted_labels = []
    
    with torch.no_grad():
        for i_batch, sample_batched in enumerate(test_loader):
            inputs, labels = sample_batched['image'], sample_batched['label']  
            inputs = inputs.to(device)
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            outputs = F.softmax(outputs)
            true_predicted_labels.append((labels.numpy(), predicted.cpu().numpy(), outputs.cpu().numpy()[0,0], outputs.cpu().numpy()[0,1]))
            correct_num+=np.sum(predicted.cpu().numpy()==labels.numpy())
            total_num+=len(labels)
            positive_correct+=np.sum(predicted.cpu().numpy()*labels.numpy())
            positive_num+=np.sum(labels.numpy())
            negative_correct+=np.sum((1-predicted.cpu().numpy())*(1-labels.numpy()))
            negative_num+=np.sum(1-labels.numpy())
            
    print('total_num:{}, test accuracy:{}, positive_acc:{}, negative_acc:{}'.format(total_num,
                                                                                   correct_num/total_num,
                                                                                    positive_correct/positive_num,
                                                                                    negative_correct/negative_num
                                                                                    ))
    return true_predicted_labels

In [59]:
#torch.save(net.state_dict(), './model/mut_clas_2019_01_15.pth')

In [60]:
net.load_state_dict(torch.load('./model/mut_clas_2019_01_21_e250_global2.pth'))

In [61]:
# print('train accuracy: ')
# Mouse_dataset = Mouse_sub_volumes(all_train_data,train_idx)
# train_dataloader = DataLoader(Mouse_dataset, batch_size=128,
#                         shuffle=False, num_workers=4)
# test(net, device, train_dataloader)

In [62]:
print('test accuracy: ')
Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

test_dic = test_with_probability(net, device, test_dataloader)

test accuracy: 


/home/zq415/pyenv/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


total_num:96, test accuracy:0.9479166666666666, positive_acc:0.9487179487179487, negative_acc:0.9444444444444444


In [63]:
cross_table = np.zeros([2,2])
mut_to_nor = []
nor_to_mul = []

for i in range(len(test_dic)):
    if test_dic[i][0] ==0 and test_dic[i][1] ==0:
        cross_table[0,0] += 1
    elif  test_dic[i][0] ==0 and test_dic[i][1] ==1:
        cross_table[0,1] += 1
        mut_to_nor.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==0:
        cross_table[1,0] += 1
        nor_to_mul.append(i)
    elif test_dic[i][0] ==1 and test_dic[i][1] ==1:
        cross_table[1,1] += 1
print(cross_table)
print(mut_to_nor)
print(nor_to_mul)

[[ 17.   1.]
 [  4.  74.]]
[90]
[16, 32, 37, 38]


In [64]:
test_dic

[(array([0]), array([0]), 0.90518457, 0.094815448),
 (array([1]), array([1]), 6.5103113e-06, 0.99999344),
 (array([1]), array([1]), 8.3632345e-09, 1.0),
 (array([1]), array([1]), 3.5636092e-06, 0.99999642),
 (array([1]), array([1]), 3.7583898e-06, 0.99999619),
 (array([1]), array([1]), 2.0452902e-08, 1.0),
 (array([1]), array([1]), 4.933419e-09, 1.0),
 (array([1]), array([1]), 4.1862616e-08, 1.0),
 (array([1]), array([1]), 9.2822479e-11, 1.0),
 (array([0]), array([0]), 0.97405595, 0.025944086),
 (array([1]), array([1]), 2.2592277e-12, 1.0),
 (array([1]), array([1]), 1.9051833e-05, 0.99998093),
 (array([1]), array([1]), 1.0120527e-05, 0.99998987),
 (array([1]), array([1]), 4.7869928e-08, 1.0),
 (array([1]), array([1]), 1.0308761e-07, 0.99999988),
 (array([1]), array([1]), 1.0540753e-07, 0.99999988),
 (array([1]), array([0]), 0.55049348, 0.44950652),
 (array([1]), array([1]), 3.1962605e-07, 0.99999964),
 (array([1]), array([1]), 0.00016167587, 0.99983835),
 (array([1]), array([1]), 1.378

In [65]:
for i in mut_to_nor:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
    img_save_data_path = './img/mul_img{}_cam.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

90


In [66]:
for i in nor_to_mul:
    print(i)
    img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
    img_save_data_path = './img/nor_img{}_cam.nii'.format(i)
    nib.save(img_nft,img_save_data_path)

16
32
37
38


In [67]:
for i in range(len(test_data)):
    if test_data[i][1] == 0:
        img_nft = nib.Nifti1Image(np.squeeze(test_data[i][0]+0.5),np.eye(4))
        img_save_data_path = './img/mul_img{}.nii'.format(i)
        nib.save(img_nft,img_save_data_path)

In [68]:
def compute_saliency_maps(X, y, model):
    """
    Compute a class saliency map using the model for images X and labels y.

    Input:
    - X: Input images; Tensor of shape (N, 3, H, W)
    - y: Labels for X; LongTensor of shape (N,)
    - model: A pretrained CNN that will be used to compute the saliency map.

    Returns:
    - saliency: A Tensor of shape (N, H, W) giving the saliency maps for the input
    images.
    """
    # Make sure the model is in "test" mode
    model.eval()
    
    # Make input tensor require gradient
    X.requires_grad_()
    ##############################################################################
    # Perform a forward and backward pass through the model to compute the gradient 
    # of the correct class score with respect to each input image. You first want 
    # to compute the loss over the correct scores (we'll combine losses across a batch
    # by summing), and then compute the gradients with a backward pass.
    ##############################################################################
    scores = model(X)
    
    # Get the correct class computed scores.
    scores = scores.gather(1, y.view(-1, 1)).squeeze()  
    
    # Backward pass, need to supply initial gradients of same tensor shape as scores.
    scores.backward(torch.tensor(10.0).cuda(device))
    
    # Get gradient for image.
    saliency = X.grad.data
    
    # Convert from 3d to 1d.
    saliency = saliency.abs()
    saliency = saliency.squeeze()
    ##############################################################################
    return saliency

In [69]:
test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']  
    inputs = inputs.to(device)
    labels = labels.to(device)
    saliency = compute_saliency_maps(inputs, labels, net)
    
    max_value = torch.max(saliency)
    saliency[saliency >= (max_value*0.2)] = 1
    saliency[saliency < (max_value*0.2)] = 0
    
    img_nft = nib.Nifti1Image(np.squeeze(inputs.cpu().detach().numpy()+0.5),np.eye(4))
    img_save_data_path = './saliency_map/img_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(img_nft,img_save_data_path)
    
    saliency_nft = nib.Nifti1Image(np.squeeze(saliency.cpu().numpy()),np.eye(4))
    saliency_save_data_path = './saliency_map/salency_label{}_{}.nii'.format(labels.cpu().numpy()[0], i_batch)
    nib.save(saliency_nft,saliency_save_data_path)
        

In [ ]:
torch.max(saliency).cpu()

In [ ]:
max_value = torch.max(saliency)
saliency[saliency >= (max_value*0.2)] = 1
saliency[saliency < (max_value*0.2)] = 0

In [51]:
# net.fc1.weight.data

torch.Size([2, 96])

In [54]:
# outputs= []
# def hook(module, input, output):
#     outputs.append(output)

# net.conv8_bn.register_forward_hook(hook)
# out = net(res)
# out = net(res1)
# print(outputs)


# test_Mouse_dataset = Mouse_sub_volumes(test_data)
# test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

# for i_batch, sample_batched in enumerate(test_dataloader):
#     inputs, labels = sample_batched['image'], sample_batched['label']  
#     inputs = inputs.to(device)
#     labels = labels.to(device)
#     saliency = compute_saliency_maps(inputs, labels, net)

SyntaxError: can't use starred expression here (cell_name, line 4)

In [56]:
net.cpu()
net.eval()

fc_weight = net.fc1.weight.data

res50_conv = nn.Sequential(*list(net.children())[:-3])
for param in res50_conv.parameters():
    param.requires_grad = False

test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']  
    saliency = compute_cam_maps(inputs, labels, net, fc_weight, res50_conv)
    
#     max_value = np.max(saliency)
#     saliency[saliency >= (max_value*0.2)] = 1
#     saliency[saliency < (max_value*0.2)] = 0
    
    img_nft = nib.Nifti1Image(np.squeeze(inputs.numpy()+0.5),np.eye(4))
    img_save_data_path = './cam_map/img_label{}_{}.nii'.format(labels.numpy()[0], i_batch)
    nib.save(img_nft,img_save_data_path)
    
    saliency_nft = nib.Nifti1Image(np.squeeze(saliency),np.eye(4))
    saliency_save_data_path = './cam_map/salency_label{}_{}.nii'.format(labels.numpy()[0], i_batch)
    nib.save(saliency_nft,saliency_save_data_path)
    

In [54]:
max_value

213477.34

In [55]:
# cam represent class saliency map
def compute_cam_maps(X, y, model, fc_weight, feature_extract): 
    model.eval()
    
    outputs = feature_extract(X).squeeze()
    channels = outputs.shape[0]
    saliency = outputs[0,...] * fc_weight[y, 0]
    for i in range(1,channels):
        saliency += outputs[i,...] * fc_weight[y, i]
    saliency = zoom(saliency.numpy(), 8)
    
    saliency = saliency - np.min(saliency)
    saliency = saliency / np.max(saliency)
    
    return saliency

In [47]:
for i in range(1,96):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95


In [76]:
net.cpu()
net.eval()

fc_weight = net.fc1.weight.data

res50_conv = nn.Sequential(*list(net.children())[:-2])
for param in res50_conv.parameters():
    param.requires_grad = False

test_Mouse_dataset = Mouse_sub_volumes(test_data)
test_dataloader = DataLoader(test_Mouse_dataset, batch_size=1, shuffle=False, num_workers=4)

for i_batch, sample_batched in enumerate(test_dataloader):
    inputs, labels = sample_batched['image'], sample_batched['label']
    print(res50_conv(inputs).shape)

torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1, 96, 1, 1, 1])
torch.Size([1,